In [1]:
import pandas as pd
from pybatfish.client.session import Session
from pybatfish.datamodel import *

pd.set_option("display.width", 300) 
pd.set_option("display.max_columns", 30) 
pd.set_option("display.max_rows", 1000) 
pd.set_option("display.max_colwidth", None)

# Configure all pybatfish loggers to use WARN level
import logging
logging.getLogger('pybatfish').setLevel(logging.WARN)

In [2]:
bf = Session(host="localhost")



#### Routing and Forwarding Tables

This category of questions allows you to query the RIBs and FIBs computed
by Batfish.


* [Routes](#Routes)
* [BGP RIB](#BGP-RIB)
* [EVPN RIB](#EVPN-RIB)
* [Longest Prefix Match](#Longest-Prefix-Match)

In [3]:
bf.set_network('generate_questions')

'generate_questions'

In [4]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Routes

Returns routing tables.

Shows routes for specified RIB, VRF, and node(s).

###### **Inputs**

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Return routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Return routes for networks matching this prefix. | str | True | 
prefixMatchType | Use this prefix matching criterion: EXACT, LONGEST_PREFIX_MATCH, LONGER_PREFIXES, SHORTER_PREFIXES. | str | True | EXACT
protocols | Return routes for protocols matching this specifier. | [RoutingProtocolSpec](../specifiers.md#routing-protocol-specifier) | True | 
vrfs | Return routes on VRFs matching this name or regex. | str | True | 
rib | Only return routes from a given protocol RIB. | str | True | 

###### **Invocation**

In [5]:
result = bf.q.routes().answer().frame()

###### **Return Value**

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Next_Hop | Route's Next Hop | [NextHop](../datamodel.rst#pybatfish.datamodel.route.NextHop)
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
Metric | Route's Metric | int
Admin_Distance | Route's Admin distance | int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [6]:
result.head(5)

,Node,VRF,Network,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,Metric,Admin_Distance,Tag
0,as1border1,default,1.0.1.0/24,interface GigabitEthernet0/0,AUTO/NONE(-1l),GigabitEthernet0/0,connected,0,0,None
1,as1border1,default,1.0.1.1/32,interface GigabitEthernet0/0,AUTO/NONE(-1l),GigabitEthernet0/0,local,0,0,None
2,as1border1,default,1.0.2.0/24,interface GigabitEthernet0/0 ip 1.0.1.2,1.0.1.2,GigabitEthernet0/0,ospf,2,110,None
3,as1border1,default,1.1.1.1/32,interface Loopback0,AUTO/NONE(-1l),Loopback0,connected,0,0,None
4,as1border1,default,1.2.2.2/32,interface GigabitEthernet0/0 ip 1.0.1.2,1.0.1.2,GigabitEthernet0/0,ospf,3,110,None


Print the first row of the returned Dataframe

In [7]:
result.iloc[0]

Node                                    as1border1
VRF                                        default
Network                                 1.0.1.0/24
Next_Hop              interface GigabitEthernet0/0
Next_Hop_IP                         AUTO/NONE(-1l)
Next_Hop_Interface              GigabitEthernet0/0
Protocol                                 connected
Metric                                           0
Admin_Distance                                   0
Tag                                           None
Name: 0, dtype: object

In [8]:
bf.set_network('generate_questions')

'generate_questions'

In [9]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### BGP RIB

Returns routes in the BGP RIB.

Shows BGP routes for specified VRF and node(s). This question is not available in Batfish containers on dockerhub prior to March 29, 2021.

###### **Inputs**

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
prefixMatchType | Use this prefix matching criterion: EXACT, LONGEST_PREFIX_MATCH, LONGER_PREFIXES, SHORTER_PREFIXES. | str | True | EXACT
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 
status | Examine routes whose status matches this specifier. | [BgpRouteStatusSpec](../specifiers.md#bgp-route-status-specifier) | True | 

###### **Invocation**

In [10]:
result = bf.q.bgpRib().answer().frame()

###### **Return Value**

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Status | Route's statuses | List of str
Next_Hop | Route's Next Hop | [NextHop](../datamodel.rst#pybatfish.datamodel.route.NextHop)
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
AS_Path | Route's AS path | str
Metric | Route's Metric | int
Local_Pref | Route's Local Preference | int
Communities | Route's List of communities | List of str
Origin_Protocol | Route's Origin protocol | str
Origin_Type | Route's Origin type | str
Originator_Id | Route's Originator ID | str
Received_From_IP | IP of the neighbor who sent this route | str
Received_Path_Id | Route's Received Path ID | int
Cluster_List | Route's Cluster List | Set of int
Tunnel_Encapsulation_Attribute | Route's BGP Tunnel Encapsulation Attribute | str
Weight | Route's BGP Weight | int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [11]:
result.head(5)

,Node,VRF,Network,Status,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Received_From_IP,Received_Path_Id,Cluster_List,Tunnel_Encapsulation_Attribute,Weight,Tag
0,as1border1,default,1.0.1.0/24,['BEST'],discard,AUTO/NONE(-1l),null_interface,bgp,,0,100,[],connected,igp,1.1.1.1,0.0.0.0,None,None,None,32768,None
1,as1border1,default,1.0.2.0/24,['BEST'],ip 1.0.1.2,1.0.1.2,dynamic,bgp,,2,100,[],ospf,igp,1.1.1.1,0.0.0.0,None,None,None,32768,None
2,as1border1,default,2.128.0.0/16,['BEST'],ip 10.12.11.2,10.12.11.2,dynamic,bgp,2,50,350,['2:1'],bgp,igp,2.1.1.1,10.12.11.2,None,None,None,0,None
3,as1border1,default,3.0.1.0/24,['BEST'],ip 10.13.22.3,10.13.22.3,dynamic,ibgp,3,50,350,['3:1'],ibgp,igp,1.2.2.2,1.10.1.1,1,[17432833],None,0,None
4,as1border1,default,3.0.2.0/24,['BEST'],ip 10.13.22.3,10.13.22.3,dynamic,ibgp,3,50,350,['3:1'],ibgp,igp,1.2.2.2,1.10.1.1,1,[17432833],None,0,None


Print the first row of the returned Dataframe

In [12]:
result.iloc[0]

Node                                  as1border1
VRF                                      default
Network                               1.0.1.0/24
Status                                  ['BEST']
Next_Hop                                 discard
Next_Hop_IP                       AUTO/NONE(-1l)
Next_Hop_Interface                null_interface
Protocol                                     bgp
AS_Path                                         
Metric                                         0
Local_Pref                                   100
Communities                                   []
Origin_Protocol                        connected
Origin_Type                                  igp
Originator_Id                            1.1.1.1
Received_From_IP                         0.0.0.0
Received_Path_Id                            None
Cluster_List                                None
Tunnel_Encapsulation_Attribute              None
Weight                                     32768
Tag                 

In [13]:
bf.set_network('generate_questions')

'generate_questions'

In [14]:
bf.set_snapshot('aristaevpn')

'aristaevpn'

##### EVPN RIB

Returns routes in the EVPN RIB.

Shows EVPN routes for specified VRF and node(s). This question is not available in Batfish containers on dockerhub prior to March 29, 2021.

###### **Inputs**

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | 
network | Examine routes for networks matching this prefix. | str | True | 
prefixMatchType | Use this prefix matching criterion: EXACT, LONGEST_PREFIX_MATCH, LONGER_PREFIXES, SHORTER_PREFIXES. | str | True | EXACT
vrfs | Examine routes on VRFs matching this name or regex. | str | True | 

###### **Invocation**

In [15]:
result = bf.q.evpnRib().answer().frame()

###### **Return Value**

Name | Description | Type
--- | --- | ---
Node | Node | str
VRF | VRF name | str
Network | Network for this route | str
Status | Route's statuses | List of str
Route_Distinguisher | Route distinguisher | str
Next_Hop | Route's Next Hop | [NextHop](../datamodel.rst#pybatfish.datamodel.route.NextHop)
Next_Hop_IP | Route's Next Hop IP | str
Next_Hop_Interface | Route's Next Hop Interface | str
Protocol | Route's Protocol | str
AS_Path | Route's AS path | str
Metric | Route's Metric | int
Local_Pref | Route's Local Preference | int
Communities | Route's List of communities | List of str
Origin_Protocol | Route's Origin protocol | str
Origin_Type | Route's Origin type | str
Originator_Id | Route's Originator ID | str
Received_Path_Id | Route's Received Path ID | int
Cluster_List | Route's Cluster List | Set of int
Tunnel_Encapsulation_Attribute | Route's BGP Tunnel Encapsulation Attribute | str
Weight | Route's BGP Weight | int
Tag | Tag for this route | int

Print the first 5 rows of the returned Dataframe

In [16]:
result.head(5)

,Node,VRF,Network,Status,Route_Distinguisher,Next_Hop,Next_Hop_IP,Next_Hop_Interface,Protocol,AS_Path,Metric,Local_Pref,Communities,Origin_Protocol,Origin_Type,Originator_Id,Received_Path_Id,Cluster_List,Tunnel_Encapsulation_Attribute,Weight,Tag
0,dc1-bl1a,default,10.1.10.0/24,['BEST'],192.168.255.3:15001,vni 15001 vtep 192.168.254.3,AUTO/NONE(-1l),dynamic,bgp,65001 65101,0,100,['2:15001:15001'],bgp,igp,192.168.255.1,None,None,None,0,None
1,dc1-bl1a,default,10.1.10.0/24,['BEST'],192.168.255.5:15001,vni 15001 vtep 192.168.254.4,AUTO/NONE(-1l),dynamic,bgp,65001 65102,0,100,['2:15001:15001'],bgp,igp,192.168.255.1,None,None,None,0,None
2,dc1-bl1a,default,10.1.10.0/24,['BEST'],192.168.255.4:15001,vni 15001 vtep 192.168.254.4,AUTO/NONE(-1l),dynamic,bgp,65001 65102,0,100,['2:15001:15001'],bgp,igp,192.168.255.1,None,None,None,0,None
3,dc1-bl1a,default,10.1.11.0/24,['BEST'],192.168.255.4:15001,vni 15001 vtep 192.168.254.4,AUTO/NONE(-1l),dynamic,bgp,65001 65102,0,100,['2:15001:15001'],bgp,igp,192.168.255.1,None,None,None,0,None
4,dc1-bl1a,default,10.1.11.0/24,['BEST'],192.168.255.5:15001,vni 15001 vtep 192.168.254.4,AUTO/NONE(-1l),dynamic,bgp,65001 65102,0,100,['2:15001:15001'],bgp,igp,192.168.255.1,None,None,None,0,None


Print the first row of the returned Dataframe

In [17]:
result.iloc[0]

Node                                                  dc1-bl1a
VRF                                                    default
Network                                           10.1.10.0/24
Status                                                ['BEST']
Route_Distinguisher                        192.168.255.3:15001
Next_Hop                          vni 15001 vtep 192.168.254.3
Next_Hop_IP                                     AUTO/NONE(-1l)
Next_Hop_Interface                                     dynamic
Protocol                                                   bgp
AS_Path                                            65001 65101
Metric                                                       0
Local_Pref                                                 100
Communities                                  ['2:15001:15001']
Origin_Protocol                                            bgp
Origin_Type                                                igp
Originator_Id                                    192.16

In [18]:
bf.set_network('generate_questions')

'generate_questions'

In [19]:
bf.set_snapshot('generate_questions')

'generate_questions'

##### Longest Prefix Match

Returns routes that are longest prefix match for a given IP address.

Return longest prefix match routes for a given IP in the RIBs of specified nodes and VRFs.

###### **Inputs**

Name | Description | Type | Optional | Default Value
--- | --- | --- | --- | --- 
ip | IP address to run LPM on. | str | False | 
nodes | Examine routes on nodes matching this specifier. | [NodeSpec](../specifiers.md#node-specifier) | True | .*
vrfs | Examine routes on VRFs matching this name or regex. | str | True | .*

###### **Invocation**

In [20]:
result = bf.q.lpmRoutes(ip='2.34.201.10').answer().frame()

###### **Return Value**

Name | Description | Type
--- | --- | ---
Node | Node where the route is present | str
VRF | VRF where the route is present | str
Ip | IP that was being matched on | str
Network | The longest-prefix network that matched | str
Num_Routes | Number of routes that matched (in case of ECMP) | int

Print the first 5 rows of the returned Dataframe

In [21]:
result.head(5)

,Node,VRF,Ip,Network,Num_Routes
0,as2border1,default,2.34.201.10,2.34.201.0/24,2
1,as2border2,default,2.34.201.10,2.34.201.0/24,2
2,as2core1,default,2.34.201.10,2.34.201.0/24,1
3,as2core2,default,2.34.201.10,2.34.201.0/24,1
4,as2dept1,default,2.34.201.10,2.34.201.0/24,1


Print the first row of the returned Dataframe

In [22]:
result.iloc[0]

Node             as2border1
VRF                 default
Ip              2.34.201.10
Network       2.34.201.0/24
Num_Routes                2
Name: 0, dtype: object